In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime
import random

# Setup WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument(f'user-agent={random.choice(["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36","Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0","Mozilla/5.0 (Macintosh; Intel Mac OS X 11.5; rv:90.0) Gecko/20100101 Firefox/90.0"])}')

def get_forex_factory_calendar():
    url = 'https://www.forexfactory.com/calendar'
    driver = create_driver()
    driver.get(url)
    calendar_table = driver.find_element(By.CLASS_NAME, "calendar__table")
    calendar_data = []

    for row in calendar_table.find_elements(By.TAG_NAME, "tr"):
        row_data = [td.text for td in row.find_elements(By.TAG_NAME, "td")]
        if row_data:
            calendar_data.append(row_data)
    driver.quit()
    return calendar_data

calendar_data = get_forex_factory_calendar()
filtered_data = [[i for i in entry if i != ''] for entry in calendar_data]
columns = ["Time", "Currency", "Impact", "Event", "Actual", "Forecast", "Previous"] 
df = pd.DataFrame(filtered_data, columns=columns)
df = df.dropna(how='all')
current_date = datetime.now().strftime("%b %d %Y")
df['Highlight'] = df['Time'].apply(lambda x: 'highlight' if x and current_date in x else '')
df_highlighted = df.style.apply(lambda x: ['background-color: yellow' if x['Highlight'] == 'highlight' else '' for _ in x], axis=1)

df_highlighted

,Time,Currency,Impact,Event,Actual,Forecast,Previous,Highlight
2,Sun Aug 18,6:30pm,NZD,BusinessNZ Services Index,44.6,40.7,None,
3,7:01pm,GBP,Rightmove HPI m/m,-1.5%,-0.4%,None,None,
4,7:50pm,JPY,Core Machinery Orders m/m,2.1%,0.9%,-3.2%,None,
6,Mon Aug 19,9:15am,USD,FOMC Member Waller Speaks,None,None,None,
7,10:00am,USD,CB Leading Index m/m,-0.6%,-0.4%,-0.2%,None,
8,6:45pm,NZD,Trade Balance,-963M,331M,585M,None,
9,9:03pm,CNY,1-y Loan Prime Rate,3.35%,3.35%,3.35%,None,
10,CNY,5-y Loan Prime Rate,3.85%,3.85%,3.85%,None,None,
11,9:30pm,AUD,Monetary Policy Meeting Minutes,None,None,None,None,
13,Tue Aug 20,2:00am,CHF,Trade Balance,4.89B,5.44B,6.12B,
